<a href="https://colab.research.google.com/github/YaCpotato/python/blob/master/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

やること  
* XGBoost理論調査
* LightGBM CatBoostとの比較

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd drive/'My Drive'/'Colab Notebooks'/

In [3]:
import pandas as pd
import io
train = pd.read_csv('./poker-hand-training-true.csv', header=None)
test = pd.read_csv('./poker-hand-testing.csv', header=None)
train.head()
test.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,1,1,13,2,4,2,3,1,12,0
1,3,12,3,2,3,11,4,5,2,5,1
2,1,9,4,6,1,4,3,2,3,9,1
3,1,4,3,13,2,13,2,1,3,6,1
4,3,10,2,7,1,2,2,11,4,9,0


こういうデータに対して(どういうデータか説明もできない)するべき前処理が未だうかばない筆者です。アドバイス求む。  
PandasのDataFrameも使おうとしたがまだ機械学習のいちプロセスで使い通せない段階

# データを整形する
* ラベルデータの取り出し(正確には代入してから削除してる)
* 学習データ、テストデータの分割

In [4]:
from sklearn import preprocessing
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import normalize

train_Y = train[:][10]
test_Y = test[:][10]

X_train, X_test, = train_test_split(train,train_size=0.7)
Y_train,Y_test = train_test_split(train_Y,train_size=0.7)
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

X_train.drop(10,axis=1,inplace=True)
X_test.drop(10,axis=1,inplace=True)
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

Using TensorFlow backend.


(17507, 11) (17507,) (7503, 11) (7503,)
(17507, 10) (17507,) (7503, 10) (7503,)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


## 学習データからValidation用にデータを取り出す

In [5]:
import numpy as np
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
Y_train = Y_train.astype('float32')
Y_test = Y_test.astype('float32')

X_train, X_val,= train_test_split(X_train,train_size=0.7)
Y_train, Y_val = train_test_split(Y_train,train_size=0.7)
print(X_train.shape,Y_train.shape,X_val.shape,Y_val.shape)

(12254, 10) (12254,) (5253, 10) (5253,)


## XGBoost　モデルを定義

In [0]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [0]:
clf = xgb.XGBClassifier()

@param max_depth: 決定木の葉の数

In [0]:
clf_cv = GridSearchCV(clf, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)

In [11]:
%%time
clf_cv.fit(X_train, Y_train,verbose=1)
# 改めて最適パラメータで学習
clf = xgb.XGBClassifier(**clf_cv.best_params_)
clf.fit(X_train, Y_train,
        early_stopping_rounds=100,
        eval_set=[[X_val, Y_val]],
        verbose=1)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.9min finished


[0]	validation_0-merror:0.505045
Will train until validation_0-merror hasn't improved in 100 rounds.
[1]	validation_0-merror:0.505045
[2]	validation_0-merror:0.505045
[3]	validation_0-merror:0.505045
[4]	validation_0-merror:0.505045
[5]	validation_0-merror:0.505045
[6]	validation_0-merror:0.505045
[7]	validation_0-merror:0.505045
[8]	validation_0-merror:0.505045
[9]	validation_0-merror:0.505045
[10]	validation_0-merror:0.505045
[11]	validation_0-merror:0.505045
[12]	validation_0-merror:0.505045
[13]	validation_0-merror:0.505045
[14]	validation_0-merror:0.505045
[15]	validation_0-merror:0.505045
[16]	validation_0-merror:0.505045
[17]	validation_0-merror:0.505045
[18]	validation_0-merror:0.505045
[19]	validation_0-merror:0.505045
[20]	validation_0-merror:0.504854
[21]	validation_0-merror:0.504854
[22]	validation_0-merror:0.504854
[23]	validation_0-merror:0.504854
[24]	validation_0-merror:0.505045
[25]	validation_0-merror:0.504854
[26]	validation_0-merror:0.504664
[27]	validation_0-merror

In [12]:
# 学習モデルの評価
from sklearn.metrics import confusion_matrix,classification_report
pred = clf.predict(X_test)
print (confusion_matrix(Y_test, pred))
print (classification_report(Y_test, pred))

[[3729   10    0    0    0    0    0    0    0    0]
 [3201    6    0    0    0    0    0    0    0    0]
 [ 343    1    0    0    0    0    0    0    0    0]
 [ 146    0    0    0    0    0    0    0    0    0]
 [  33    0    0    0    0    0    0    0    0    0]
 [  19    0    0    0    0    0    0    0    0    0]
 [  11    0    0    0    0    0    0    0    0    0]
 [   2    0    0    0    0    0    0    0    0    0]
 [   1    0    0    0    0    0    0    0    0    0]
 [   1    0    0    0    0    0    0    0    0    0]]
              precision    recall  f1-score   support

         0.0       0.50      1.00      0.66      3739
         1.0       0.35      0.00      0.00      3207
         2.0       0.00      0.00      0.00       344
         3.0       0.00      0.00      0.00       146
         4.0       0.00      0.00      0.00        33
         5.0       0.00      0.00      0.00        19
         6.0       0.00      0.00      0.00        11
         7.0       0.00      0.00   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
